   # Vaccination rate in schools - where can we improve?
   
   <img src=vaccine.jpg width="900">
   
   **Credit:**  [healthline](https://www.healthline.com/health-news/vaccinations-before-new-school-year) 

In [1]:
# Load relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sm
import warnings

warnings.filterwarnings("ignore")  # Suppress all warnings

## Introduction

**Business Context:** Reporters at the Wall Street Journal collected data on school-specific vaccination rates. In total, the WSJ’s dataset covers more than 46,000 schools, of which 42,000 have at least one vaccination rate available. Most states provided data for the 2018–19 school year. There is also data from the Census that estimates population and poverty by district, and median household income per state.  

**Analytics Context:**

Questions: 
1. What are the states with higher and lower vaccination rates? 
2. Does socioeconomic status play any role in vaccination rate?

**Goal**: Create a model to predict vaccination compliance at schools in the United States.


## Data Wrangling
### Extracting and cleaning relevant data

Let's start looking at the datasets.

In [2]:
# vaccination datset containing vaccination rate for schools by county/district
vaccine_df = pd.read_csv('state-overviews.csv', index_col=0)
vaccine_df = vaccine_df.sort_values(by=['state','county/district'], ascending=True)
vaccine_df.head()


,state,year,county/district,enroll,mmr,overall,xmed,xper,xrel
index,,,,,,,,,
1,Alabama,2017-18,Autauga,1817,64.17,96.39,0.04,NaN,0.57
2,Alabama,2017-18,Baldwin,5479,70.89,96.53,0.09,NaN,1.15
3,Alabama,2017-18,Barbour,733,72.17,88.27,0.05,NaN,0.13
4,Alabama,2017-18,Bibb,538,66.54,94.54,0,NaN,0.54
5,Alabama,2017-18,Blount,1450,70.69,97.3,0,NaN,0.46


In [3]:
# updating state long name to abbreviation version
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

vaccine_df['state'] = vaccine_df['state'].map(us_state_abbrev)
vaccine_df.rename(columns={'county/district': 'district name'}, inplace=True)
vaccine_df

,state,year,district name,enroll,mmr,overall,xmed,xper,xrel
index,,,,,,,,,
1,AL,2017-18,Autauga,1817,64.17,96.39,0.04,NaN,0.57
2,AL,2017-18,Baldwin,5479,70.89,96.53,0.09,NaN,1.15
3,AL,2017-18,Barbour,733,72.17,88.27,0.05,NaN,0.13
4,AL,2017-18,Bibb,538,66.54,94.54,0,NaN,0.54
5,AL,2017-18,Blount,1450,70.69,97.3,0,NaN,0.46
...,...,...,...,...,...,...,...,...,...
19,WY,2018-19,Sweetwater,616,80.00,75,NaN,NaN,NaN
20,WY,2018-19,Teton,218,72.00,65,NaN,NaN,NaN
21,WY,2018-19,Uinta,348,72.00,64,NaN,NaN,NaN


In [4]:
counties_for_state = {}

for state in vaccine_df['state'].unique():
    counties_in_state = vaccine_df[vaccine_df['state'] == state]['district name'].values.tolist()
    counties_for_state[state] = [x for x in counties_in_state]

# Add empty list for states not represented in the other data set
for state in ['AK', 'AR', 'DE', 'DC', 'GA', 'HI', 'ID', 'IL', 'MO', 'MS', 'NH', 'PR', 'VA', 'WV']:
    counties_for_state[state] = []

counties_for_state

{'AL': ['Autauga',
  'Baldwin',
  'Barbour',
  'Bibb',
  'Blount',
  'Bullock',
  'Butler',
  'Calhoun',
  'Chambers',
  'Cherokee',
  'Chilton',
  'Choctaw',
  'Clarke',
  'Clay',
  'Cleburne',
  'Coffee',
  'Colbert',
  'Conecuh',
  'Coosa',
  'Covington',
  'Crenshaw',
  'Cullman',
  'Dale',
  'Dallas',
  'DeKalb',
  'Elmore',
  'Escambia',
  'Etowah',
  'Fayette',
  'Franklin',
  'Geneva',
  'Greene',
  'Hale',
  'Henry',
  'Houston',
  'Jackson',
  'Jefferson',
  'Lamar',
  'Lauderdale',
  'Lawrence',
  'Lee',
  'Limestone',
  'Lowndes',
  'Macon',
  'Madison',
  'Marengo',
  'Marion',
  'Marshall',
  'Mobile',
  'Monroe',
  'Montgomery',
  'Morgan',
  'Perry',
  'Pickens',
  'Pike',
  'Randolph',
  'Russell',
  'Shelby',
  'St Clair',
  'Sumter',
  'Talladega',
  'Tallapoosa',
  'Tuscaloosa',
  'Walker',
  'Washington',
  'Wilcox',
  'Winston'],
 'AZ': ['Apache',
  'Cochise',
  'Coconino',
  'Gila',
  'Graham',
  'Greenlee',
  'La Paz',
  'Maricopa',
  'Mohave',
  'Navajo',
  'Pi

------------------

The dataset below contains estimated number of relevant children 5 to 17 years old in poverty who are related to the householder. The data has information at the district level.

In [13]:
poverty_df = pd.read_excel('poverty_rate_district18.xls', header=None)

poverty_df.drop(0, inplace=True) #dropping unnamed row 0
poverty_df.drop(1, inplace=True) # dropping row 1

new_header = poverty_df.iloc[0] #grab the first row for the header
poverty_df = poverty_df[1:] #take the data less the header row
poverty_df.columns = new_header #set the header row as the df header


In [6]:
#poverty_df['matches'] = poverty_df.apply(lambda x: any(county in x['Name'].lower() for county in counties_for_state[x['State Postal Code']]), axis=1)
#poverty_df = poverty_df[poverty_df['matches'] == True]


In [41]:
def normalized_name(state_postal_code, name):
    county = None
    for county in counties_for_state[state_postal_code]:
        if county.lower() in name.lower():
            return county
    return None

poverty_df['district name'] = poverty_df.apply(lambda x: normalized_name(x['State Postal Code'], x['Name']), axis=1)
poverty_df_clean = poverty_df[poverty_df['district name'].notna()]


In [34]:
merge_df = pd.merge(vaccine_df, poverty_df_clean, on='district name', how='inner')
merge_df

,state,year,district name,enroll,mmr,overall,xmed,xper,xrel,State Postal Code,State FIPS Code,District ID,Name,Estimated Total Population,Estimated Population 5-17,Estimated number of relevant children 5 to 17 years old in poverty who are related to the householder
0,AL,2017-18,Autauga,1817,64.17,96.39,0.04,NaN,0.57,AL,01,00240,Autauga County School District,55601,9799,1891
1,AL,2017-18,Baldwin,5479,70.89,96.53,0.09,NaN,1.15,AL,01,00270,Baldwin County School District,218022,35155,4534
2,AL,2017-18,Barbour,733,72.17,88.27,0.05,NaN,0.13,AL,01,00300,Barbour County School District,12978,1671,639
3,AL,2017-18,Bibb,538,66.54,94.54,0,NaN,0.54,AL,01,00360,Bibb County School District,22400,3302,840
4,AL,2017-18,Blount,1450,70.69,97.3,0,NaN,0.46,AL,01,00420,Blount County School District,51201,8919,1357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6692,WY,2018-19,Uinta,348,72.00,64,NaN,NaN,NaN,WY,56,04260,Uinta County School District 6,3122,730,33
6693,WY,2018-19,Washakie,117,94.00,89,NaN,NaN,NaN,WY,56,06240,Washakie County School District 1,7208,1297,183
6694,WY,2018-19,Washakie,117,94.00,89,NaN,NaN,NaN,WY,56,05820,Washakie County School District 2,677,90,8
6695,WY,2018-19,Weston,78,92.00,76,NaN,NaN,NaN,WY,56,04830,Weston County School District 1,5497,834,135


In [40]:
merge_df_grouped = merge_df.groupby(['state', 'district name']).agg({
    'District ID': lambda x: x.iloc[0], # Only keep the first row's value
    'enroll': lambda x: x.iloc[0],
    'mmr': lambda x: x.iloc[0],
    'overall': lambda x: x.iloc[0],
    'xmed': lambda x: x.iloc[0],
    'xper': lambda x: x.iloc[0],
    'xrel': lambda x: x.iloc[0],
    'Estimated Total Population': np.sum,
    'Estimated Population 5-17': np.sum,
    'Estimated number of relevant children 5 to 17 years old in poverty who are related to the householder': np.sum
})
merge_df_grouped

District ID enroll    mmr overall  xmed xper  xrel  \
state district name                                                      
AL    Autauga             00240   1817  64.17   96.39  0.04  NaN  0.57   
      Baldwin             00270   5479  70.89   96.53  0.09  NaN  1.15   
      Barbour             00300    733  72.17   88.27  0.05  NaN  0.13   
      Bibb                00360    538  66.54   94.54     0  NaN  0.54   
      Blount              00420   1450  70.69    97.3     0  NaN  0.46   
...                         ...    ...    ...     ...   ...  ...   ...   
WY    Sweetwater          05302    616  80.00      75   NaN  NaN   NaN   
      Teton               05830    218  72.00      65   NaN  NaN   NaN   
      Uinta               02760    348  72.00      64   NaN  NaN   NaN   
      Washakie            06240    117  94.00      89   NaN  NaN   NaN   
      Weston              04830     78  92.00      76   NaN  NaN   NaN   

                     Estimated Total Population  Estimated Population 5-17  \
state district name                                                          
AL    Autauga                             55601                       9799   
      Baldwin                            218022                      35155   
      Barbour                             12978                       1671   
      Bibb                                22400                       3302   
      Blount                             144308                      22543   
...                                         ...                        ...   
WY    Sweetwater                          42947                       8329   
      Teton                               23081                       3107   
      Uinta                               20299                       4358   
      Washakie                             7885                       1387   
      Weston                               6967                       1088   

                     Estimated number of relevant children 5 to 17 years old in poverty who are related to the householder  
state district name                                                                                                         
AL    Autauga                                                     1891                                                      
      Baldwin                                                     4534                                                      
      Barbour                                                      639                                                      
      Bibb                                                         840                                                      
      Blount                                                      3182                                                      
...                                                                ...                                                      
WY    Sweetwater                                                   697                                                      
      Teton                                                        180                                                      
      Uinta                                                        454                                                      
      Washakie                                                     191                                                      
      Weston                                                       150                                                      

[2125 rows x 10 columns]

-------------

In [76]:
# dataset containing percentage of students with high school diploma or higher
# read csv file
file = "Educational Attainment Percent high school graduate or higher by State.csv"
highschoolgrad_df = pd.read_csv(file)

new_header1 = highschoolgrad_df.iloc[0] #grab the first row for the header
highschoolgrad_df = highschoolgrad_df[1:] #take the data less the header row
highschoolgrad_df.columns = new_header1 #set the header row as the df header
highschoolgrad_df = highschoolgrad_df.drop(highschoolgrad_df.index[-2:]) # drop last 2 rows

highschoolgrad_df['state'] = highschoolgrad_df['State'].map(us_state_abbrev) # updating long state name to abbreviation form
highschoolgrad_df.rename(columns={'Education': '% HS graduate or higher'}, inplace=True) # renaming column
del highschoolgrad_df['Margin Of Error'] # deleting colummn margin of error



In [77]:
df = pd.merge(merge_df_grouped, highschoolgrad_df, on='state', how='left')
del df['State'] # deleting colummn State
df

,state,District ID,enroll,mmr,overall,xmed,xper,xrel,Estimated Total Population,Estimated Population 5-17,Estimated number of relevant children 5 to 17 years old in poverty who are related to the householder,% HS graduate or higher
0,AL,00240,1817,64.17,96.39,0.04,NaN,0.57,55601,9799,1891,86.2%
1,AL,00270,5479,70.89,96.53,0.09,NaN,1.15,218022,35155,4534,86.2%
2,AL,00300,733,72.17,88.27,0.05,NaN,0.13,12978,1671,639,86.2%
3,AL,00360,538,66.54,94.54,0,NaN,0.54,22400,3302,840,86.2%
4,AL,00420,1450,70.69,97.3,0,NaN,0.46,144308,22543,3182,86.2%
...,...,...,...,...,...,...,...,...,...,...,...,...
2120,WY,05302,616,80.00,75,NaN,NaN,NaN,42947,8329,697,93.2%
2121,WY,05830,218,72.00,65,NaN,NaN,NaN,23081,3107,180,93.2%
2122,WY,02760,348,72.00,64,NaN,NaN,NaN,20299,4358,454,93.2%
2123,WY,06240,117,94.00,89,NaN,NaN,NaN,7885,1387,191,93.2%


----------

In [88]:
# dataset containing percentage of students with high school diploma or higher
# read csv file
#politicalparty_df = pd.read_csv('usa-2016-presidential-election-by-county.csv')
#politicalparty_df